# Logging

We configure how logging messages should be displayed and which log level should be used before importing Haystack. Example log message: INFO - haystack.utils.preprocessing - Converting data/tutorial1/218_Olenna_Tyrell.txt Default log level in basicConfig is WARNING so the explicit parameter is not necessary but can be changed easily:

In [1]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# Building a per-document query function

Rather than querying from a large document store, this notebook
explores the use case where queries need to be made against a
*specific* document. We'll build single-document document stores,
and then retrievers, readers, and pipelines as normal.

In [2]:
from haystack.document_stores import InMemoryDocumentStore
from haystack.utils import convert_files_to_docs
from haystack.nodes import TfidfRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline

doc_dir = 'data/theringer'

docs = convert_files_to_docs(dir_path=doc_dir, split_paragraphs=True)

INFO - haystack.utils.preprocessing -  Converting data\theringer\robert-sarver-phoenix-suns-suspension.txt
INFO - haystack.utils.preprocessing -  Converting data\theringer\small-market-teams-are-paying-big-prices-to-go-all-in.txt


In [3]:
def run_query(documents, query, params=None):

    # Create in-memory document store
    document_store = InMemoryDocumentStore()
    document_store.write_documents(documents)

    # Create retriever against document store
    retriever = TfidfRetriever(document_store=document_store)

    # Load a local model or any of the QA models on
    # Hugging Face's model hub (https://huggingface.co/models)
    reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

    # Construct an extractive QA pipeline and run the query
    pipe = ExtractiveQAPipeline(reader, retriever)
    return pipe.run(query=query, params=params)

# Voilà! Ask questions!

In [4]:
# Use a util to simplify printed output
from haystack.utils import print_answers

queries = {
    'robert-sarver-phoenix-suns-suspension.txt':
        'How long is Robert Sarver suspended for?',
    'small-market-teams-are-paying-big-prices-to-go-all-in.txt':
        'What team was Donovan Mitchell traded to?',
}

for doc_name in queries:
    documents = [doc for doc in docs if doc.meta['name'] == doc_name]
    if not documents:
        raise ValueError
    query = queries[doc_name]
    prediction = run_query(documents, query)

    # Change `minimum` to `medium` or `all` to control the level of detail
    print_answers(prediction, details="minimum")

INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.nodes.retriever.sparse -  Found 10 candidate paragraphs from 10 docs in DB
INFO - haystack.modeling.utils -  Using devices: CUDA:0
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 15 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /


Query: How long is Robert Sarver suspended for?
Answers:
[   {   'answer': 'a year',
        'context': 'and former Suns employeesâ€”was to suspend Sarver from NBA '
                   'activity for a year, fine him $10 million (a fraction of '
                   'his net worth), and require that h'},
    {   'answer': 'one-year',
        'context': 'A one-year suspension is nothing; a mandated course is '
                   'less than that. If the goal of the NBAâ€™s cursory '
                   'punishment is to change anything about what '},
    {   'answer': 'a year',
        'context': 'ated, discriminated against, and mistreated his employees, '
                   'and he takes a year hiatus and attends a seminar. The '
                   'value in any punishment comes from ei'},
    {   'answer': 'a while',
        'context': 'today, or about the prospect of taking a job with the team '
                   'in the future. The root cause is still there; it just '
                

INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: english
INFO - haystack.modeling.model.language_model -  Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA
INFO - haystack.modeling.utils -  Number of GPUs: 1
INFO - haystack.modeling.infer -  Got ya 15 parallel workers to do inference ...
INFO - haystack.modeling.infer -   0     0     0     0     0     0     0     0     0     0     0     0     0     0     0  
INFO - haystack.modeling.infer -  /w\   /w\   /w\   /w\   /w\   /w\   /w\   /|\   /w\   /w\   /w\   /w\   /w\   /w\   /|\ 
INFO - haystack.modeling.infer -  /'\   / \   /'\   /'\   / \   / \   /'\   /'\   /'\   /'\   /'\   /'\   / \   /'\   /'\ 
Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 54.99 Batches/s]



Query: What team was Donovan Mitchell traded to?
Answers:
[   {   'answer': 'Cavaliers',
        'context': 'amic, with small-market teams seizing the initiative. On '
                   'Thursday, the Cavaliers completed a stealth trade for '
                   'Donovan Mitchell, swapping three first-'},
    {   'answer': 'Jazz',
        'context': 'f this list shows how trades have evolved over the last '
                   'year or two: The Jazz take the top spot because most of '
                   'the Thunderâ€™s future stash is protec'},
    {   'answer': 'Clevelandâ€™s version of Giannis, itâ€™ll sure be nice to '
                  'already have Mitchell on the roster when he does. The '
                  'Cavaliers',
        'context': 'ley turns into Clevelandâ€™s version of Giannis, itâ€™ll '
                   'sure be nice to already have Mitchell on the roster when '
                   'he does. The Cavaliers donâ€™t need '},
    {   'answer': 'Clippers',
        'contex